# FDIC Data API Client

By 503 Team 4

Source:
https://banks.data.fdic.gov/docs/#/

## Code

Import libraries

In [8]:
import requests
import pandas as pd
import json
import urllib.parse


Define function

In [70]:
# set endpoint
endpoint = 'https://banks.data.fdic.gov/api/'

# function
def get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit=5000):
    """
    NC (Noninsured non-deposit commercial banks) are removed below to match the query of the FDIC site https://banks.data.fdic.gov/bankfind-suite/financialreporting
    """
    
    
    # initialize data list
    financials = []
    
    # query
    fields_encoded = urllib.parse.quote(fields, safe='')
    url = f'{endpoint}financials?filters=RISDATE:{reporting_period} AND !(BKCLASS:"NC")&fields={fields_encoded}&limit={result_limit}&sort_by=REPDTE&sort_order=ASC'
    
    # get response
    response = requests.get(url)

    if response.status_code == 200:
        # response to json
        data = response.json()

        # store response data
        financial_data = [item['data'] for item in data['data']]
        financials.extend(financial_data)

    else:
        print(f'Request failed with status code {response.status_code}')
        return None

    return pd.DataFrame(financials)


Query the data

In [71]:
# params
reporting_period = '2022-12-31'
fields = 'REPDTE,CERT,NAME,CITY,STNAME,ASSET,DEPDOM,INSTTYPE,BKCLASS,STCNTY,CBSA,CITYHCR' # no spaces between!
result_limit = 10000

# call
all_financial_reporting_data = get_all_financial_reporting_data(endpoint, reporting_period, fields, result_limit)

# display
if all_financial_reporting_data is not None:
    all_financial_reporting_data = all_financial_reporting_data.sort_values('CERT')
    display(all_financial_reporting_data.head())

,CITYHCR,STCNTY,CBSA,CITY,BKCLASS,INSTTYPE,REPDTE,DEPDOM,ASSET,STNAME,CERT,NAME,ID
848,BOSTON,Suffolk,"BOSTON-CAMBRIDGE-NEWTON, MA-NH",BOSTON,SM,CFR,20221231,163284000,298020000,MASSACHUSETTS,14,STATE STREET BANK&TRUST CO,14_20221231
3242,AUBURN,Lee,"AUBURN-OPELIKA, AL",AUBURN,SM,CFR,20221231,952037,1023366,ALABAMA,35,AUBURNBANK,35_20221231
3319,DEMOPOLIS,Marengo,None,DEMOPOLIS,NM,CFR,20221231,407949,444822,ALABAMA,39,ROBERTSON BANKING CO,39_20221231
3376,PHENIX CITY,Russell,"COLUMBUS, GA-AL",PHENIX CITY,NM,CFR,20221231,266874,265272,ALABAMA,41,PHENIX-GIRARD BANK,41_20221231
3491,,Conecuh,None,EVERGREEN,NM,CFR,20221231,70649,76239,ALABAMA,49,BANK OF EVERGREEN,49_20221231


Check data

In [72]:
display(all_financial_reporting_data.shape)
column_to_check = 'BKCLASS'
display(pd.DataFrame({'col1': all_financial_reporting_data[column_to_check].value_counts().index, 'count': all_financial_reporting_data[column_to_check].value_counts().values}))



(4715, 13)

,col1,count
0,NM,2743
1,N,720
2,SM,701
3,SI,254
4,SB,253
5,SL,35
6,OI,9
